1.
Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Create a path
path = r'C:\Users\pinto\OneDrive\Documenti\1-2024-Instacart Basket Analysis'

In [3]:
# Import ords_prods_merge_new.pkl
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_new.pkl'))

2.
In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [4]:
# Group dataframe by 'product_name' column
ords_prods_merge.groupby('product_name')

In [5]:
# Obtain the mean values for "order_number" column grouped by "department_id"
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


This result differs from the subset since this list is much longer than before. We have here 20 values in the "department_id" column, while before there were only 8. Moreover, mean value is different compared to the subset's one. For instance, for "department_id" = 20 the order_number mean value is 16.473447, while before was 17.599636. This means that, having more values, we are able to obtain a more precise mean.

4.
Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [6]:
# Create "max_order" column
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\pinto\AppData\Local\Temp\ipykernel_15020\3738574265.py:2: FutureWarning: The provided callable <function amax at 0x00000258425D1DA0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [7]:
# Setting the flags for customers' loyalty

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Check frequency 
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

In [12]:
# Check the accuracy of the flag labels
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


5.
Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [13]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['min', 'max', 'mean']})

prices                    
                    min      max       mean
loyalty_flag                               
Loyal customer      1.0  99999.0  10.388747
New customer        1.0  99999.0  13.294370
Regular customer    1.0  99999.0  12.496203

It seems that the price of the products purchased by new customers, on average, is a little bit higher than the ones purchased by loyal and regular customers. 

6.
Create a spending flag for each user based on the average price across all their orders using the following criteria:

1)If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

2)If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [14]:
# Create 'mean_price' column
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\pinto\AppData\Local\Temp\ipykernel_15020\2896892004.py:2: FutureWarning: The provided callable <function mean at 0x00000258425D2AC0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [15]:
# Setting the flags for spenders

In [16]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'Low spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spending_flag'] = 'High spender'

In [18]:
# Check frequency 
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64

In [19]:
# Check the accuracy of the flag labels
ords_prods_merge[['user_id', 'spending_flag', 'mean_price']].head(60)

,user_id,spending_flag,mean_price
0,1,Low spender,6.367797
1,1,Low spender,6.367797
2,1,Low spender,6.367797
3,1,Low spender,6.367797
4,1,Low spender,6.367797
5,1,Low spender,6.367797
6,1,Low spender,6.367797
7,1,Low spender,6.367797
8,1,Low spender,6.367797
9,1,Low spender,6.367797


7. 
Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [20]:
# Create 'median_days' column
ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\pinto\AppData\Local\Temp\ipykernel_15020\2730646336.py:2: FutureWarning: The provided callable <function median at 0x0000025842713A60> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [21]:
# Setting the flags for frequency

In [22]:
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [23]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 20, 'frequency_flag'] = 'Regular customer'

In [24]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [25]:
# Check frequency 
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: count, dtype: int64

In [26]:
# Check the accuracy of the flag labels
ords_prods_merge[['user_id', 'frequency_flag', 'median_days', 'days_since_prior_order']].head(60)

,user_id,frequency_flag,median_days,days_since_prior_order
0,1,Non-frequent customer,20.5,NaN
1,1,Non-frequent customer,20.5,15.0
2,1,Non-frequent customer,20.5,21.0
3,1,Non-frequent customer,20.5,29.0
4,1,Non-frequent customer,20.5,28.0
5,1,Non-frequent customer,20.5,19.0
6,1,Non-frequent customer,20.5,20.0
7,1,Non-frequent customer,20.5,14.0
8,1,Non-frequent customer,20.5,0.0
9,1,Non-frequent customer,20.5,30.0


8.
Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [27]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_grouped.pkl'))